In [82]:
import os
import json
import nibabel as nib
from nibabel.testing import data_path
import numpy as np

import pydicom
from pydicom.dataset import Dataset
import pydicom._storage_sopclass_uids

import rasterio as rio
#from rasterio.features import rasterize
from rasterio import features

from shapely.geometry import Polygon, MultiPolygon, shape, mapping


In [53]:
# from nifti path
basepath = ''
base_nii = os.path.join(basepath, 'emidec-dataset-1.0.1')

# to dicoms and annotation paths
imgs_path = os.path.join(basepath, 'Imgs')
gold_path = os.path.join(basepath, 'Gold')
# to cases
#cases_path = os.path.join(basepath, 'Cases')
#for p in [imgs_path, gold_path, cases_path]: 
#    if not os.path.exists(p): os.mkdir(p)

# load dictionary of case name --> studyuid and sopinstanceuids of slices
with open('case_slice_to_sops.json', 'r') as fp:
    sort_dict = json.load(fp)

In [54]:
def to_polygon(mask):
    """Convert mask to Polygons (Origin (0.0, 0.0))
    
    Note:
        rasterio.features.shapes(source, mask=None, connectivity=4, transform=Affine(1.0, 0.0, 0.0, 0.0, 1.0, 0.0))
        For Origin (-0.5, -0.5) apply Polygon Transformation -0.5 for all xy
        https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html#rasterio.features.shapes
        
    Args:
        mask (ndarray (2D array of np.uint8): binary mask
        
    Returns:
        MultiPolygon | Polygon: Geometries extracted from mask, empty Polygon if empty mask
    """
    polygons = []
    for geom, val in rio.features.shapes(mask):
        if val:
            polygon = shape(geom)
            if polygon.geom_type == 'Polygon' and polygon.is_valid: polygons.append(polygon)
            else: print('Ignoring GeoJSON with cooresponding shape: ' + 
                      str(polygon.geom_type) + ' | Valid: ' + str(polygon.is_valid))
    return MultiPolygon(polygons) if len(polygons)>0 else Polygon()


In [146]:
###################
# Transform Annos #
###################
def emidec_to_anno_dict(mask, img_size, pixel_size):
    # 1: endo # 2: healthy myo # 3: scar # 4: no reflow
    endo_cont     = to_polygon((mask==1).astype(np.int16))
    myo_cont      = to_polygon((mask>=2).astype(np.int16))
    scar_cont     = to_polygon((mask==3).astype(np.int16))
    noreflow_cont = to_polygon((mask==4).astype(np.int16))
    anno_dict = dict()
    p_size = [float(np.round(pixel_size[0], 6)), float(np.round(pixel_size[1], 6))]
    if not endo_cont.is_empty:     anno_dict['lv_endo']  = {'imageSize': img_size, 'pixelSize': p_size, 'cont': mapping(endo_cont)}
    if not myo_cont.is_empty :     anno_dict['lv_myo']   = {'imageSize': img_size, 'pixelSize': p_size, 'cont': mapping(myo_cont)}
    if not scar_cont.is_empty:     anno_dict['lv_scar']  = {'imageSize': img_size, 'pixelSize': p_size, 'cont': mapping(scar_cont)}
    if not noreflow_cont.is_empty: anno_dict['noreflow'] = {'imageSize': img_size, 'pixelSize': p_size, 'cont': mapping(noreflow_cont)}
    return anno_dict

def emidec_transform_to_readable_annos(nii_annos, bpath, studyiuid, sops):
    contdir = os.path.join(bpath, studyiuid) 
    if not os.path.exists(contdir): os.mkdir(contdir)
    h , w, nr_slices = nii_annos.shape
    ph, pw, slice_th = nii_annos.header['pixdim'][1:4]
    mask_data = nii_annos.get_fdata().astype(np.int32)
    ll_annos = []
    for d in range(nr_slices):
        ll_annos.append(emidec_to_anno_dict(mask_data[:,:,d], [h,w], [ph,pw]))
        with open(os.path.join(contdir, sops[d]+'.json'), 'w') as f:
            print(ll_annos[-1])
            json.dump(ll_annos[-1], f)
    return ll_annos        

In [195]:
def nifti_to_dcm(nii_imgs, bpath, casename, studyuid, sops):
    if not os.path.exists(os.path.join(bpath, casename)):
        os.mkdir(os.path.join(bpath, casename))
    h, w, nr_slices  = nii_imgs.header['dim'][1:4]
    ph, pw, pdepth = nii_imgs.header['pixdim'][1:4]
    print(nii_imgs)
    imgs = nii_imgs.get_fdata()
    ds = Dataset()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    for d in range(nr_slices):
        img = imgs[:,:,d]
        img = img.astype(np.uint16)
        meta = pydicom.Dataset()
        meta.MediaStorageSOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
        meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
        meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
        
        ds.file_meta = meta
        ds.SOPInstanceUID = sops[d]
        ds.is_little_endian = True
        ds.is_implicit_VR = False
        ds.SOPClassUID = pydicom._storage_sopclass_uids.MRImageStorage
        ds.PatientName = casename
        ds.PatientID = "123456"
        ds.Modality = "MR"
        
        ds.StudyInstanceUID  = studyuid
        ds.FrameOfReferenceUID = pydicom.uid.generate_uid()
        ds.SeriesDescription = "sax lge"
        ds.SliceLocation  = d*pdepth
        ds.SliceThickness = str(pdepth)
        ds.SpacingBetweenSlices = str(pdepth)
        ds.PixelSpacing = str(ph)+'\\'+str(pw)
        ds.SeriesNumber = 0
        ds.BitsStored = 16
        ds.BitsAllocated = 16
        ds.SamplesPerPixel = 1
        ds.HighBit = 15
        ds.ImagesInAcquisition = "1"
        ds.Rows    = h
        ds.Columns = w
        ds.InstanceNumber = 0
        ds.ImagePositionPatient = [0, 0, d*pdepth]
        ds.ImageOrientationPatient = [1, 0, 0, 0, 1, 0]
        ds.ImageType = r"ORIGINAL\PRIMARY\AXIAL"
        ds.RescaleIntercept = "0"
        ds.RescaleSlope     = "1"
        ds.PhotometricInterpretation = "MONOCHROME2"
        ds.PixelRepresentation = 1
        pydicom.dataset.validate_file_meta(ds.file_meta, enforce_standard=True)
        ds.PixelData = img.tobytes()
        ds.private_block(0x000b, 'Lumos: SAX LGE', create=True)
        filename = os.path.join(bpath, casename, str(d)+'.dcm')
        ds.save_as(filename=filename, write_like_original=False)

In [ ]:
folders = [f for f in os.listdir(base_nii) if os.path.isdir(os.path.join(base_nii, f))]
for f in folders:
    img_path  = os.path.join(base_nii, f, 'Images')
    anno_path = os.path.join(base_nii, f, 'Contours')
    nii_imgs  = nib.load(os.path.join(img_path,  f+'.nii.gz'))
    nii_annos = nib.load(os.path.join(anno_path, f+'.nii.gz'))
    studyiuid, sops = sort_dict[f]['study_uid'], sort_dict[f]['sopinstanceuids']
    nifti_to_dcm(nii_imgs, imgs_path, f, studyiuid, sops)
    annos = emidec_transform_to_readable_annos(nii_annos, gold_path, studyiuid, sops)

In [183]:
{task_id: ObjectId('66953ff3351f64c12ca179d6')} emidec gold task id

{name: "PatientX"} cases, dicoms, evals, image_orgs
{name: "EMIDEC"} cohorts

{ $and: [ { task_id: ObjectId('65bb92474e35afb663dc6dcd') , studyuid: "1.2.826.0.1.3680043.8.498.84770591399155535260156713669022621008" } ] } FWHM, P001
"1.2.826.0.1.3680043.8.498.11307161898731132853843728987059963152" P002
"1.2.826.0.1.3680043.8.498.89586328180080351709517792489268278244" P008
"1.2.826.0.1.3680043.8.498.98429712384624343332387349997390016230" P022
"1.2.826.0.1.3680043.8.498.80940209359570484935093203199410241980" P028
"1.2.826.0.1.3680043.8.498.10470484482073739411381235242276910676" P035
"1.2.826.0.1.3680043.8.498.66299984376228452257781909889448293678" P038
"1.2.826.0.1.3680043.8.498.54081579330563095952156138035623532519" P043
"1.2.826.0.1.3680043.8.498.10553469388410035014272265768844628497" P051
"1.2.826.0.1.3680043.8.498.98126826760554798030171018849046942658" P053
"1.2.826.0.1.3680043.8.498.82112234070847910019345620487810119653" P055
"1.2.826.0.1.3680043.8.498.12804984749518611674890828147381260355" P059
"1.2.826.0.1.3680043.8.498.44071527813010678852610504834211799198" P064
"1.2.826.0.1.3680043.8.498.56304682190392844280230928869543534114" P071
"1.2.826.0.1.3680043.8.498.10495728983750682277972128398573411394" P073
"1.2.826.0.1.3680043.8.498.91199922816855767043237018621627511392" P076
"1.2.826.0.1.3680043.8.498.83935206222398332717932000245464285764" P080
"1.2.826.0.1.3680043.8.498.88226697983927685486152243677897533684" P082
"1.2.826.0.1.3680043.8.498.21228448594165958302374490795200104000" P093


SyntaxError: invalid syntax (4071086460.py, line 1)